In [24]:
import yfinance as yf
import pandas as pd
from datetime import datetime

In [25]:
train_start = "2021-01-01"
train_end = "2023-12-31"
test_start = "2024-01-01"
test_end = "2024-12-31"
forecast_start = "2025-01-01"
forecast_end = "2025-04-30"

In [26]:
Main_tickers = [
    'AAPL', 'MSFT', 'NVDA', 'ORCL', 'ADBE', 'CSCO', 'CRM', 'INTC', 'AMD', 'QCOM', 'TXN', 'IBM', 'NOW', 'AMAT', 'LRCX',
    'MU', 'PANW', 'SNPS', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'C', 'BLK', 'AXP', 'SCHW', 'SPGI', 'MA', 'V', 'T', 'VZ', 'CMCSA',
    'MMC', 'JNJ', 'LLY', 'PFE', 'MRK', 'ABBV', 'TMO', 'DHR', 'AMGN', 'GILD', 'CVS', 'UNH', 'ELV', 'CI', 'BMY', 'ZTS', 'A', 'TSLA',
    'HD', 'MCD', 'NKE', 'SBUX', 'LOW', 'BKNG', 'TX', 'GM', 'F', 'YUM', 'EBAY', 'DHI', 'CCI', 'PG', 'KO', 'PEP', 'WBA', 'CL', 'PM',
    'MDLZ', 'KMB', 'KHC', 'CAT', 'GPS', 'BA', 'DE', 'HON', 'UNP', 'MMM', 'GE', 'LMT', 'RTX', 'XOM', 'CVX', 'SLB', 'OXY',
    'GOOGL', 'META', 'DIS', 'LIN', 'SHW', 'FCX', 'ECL', 'NEM', 'NEE', 'DUK', 'SO', 'AMT', 'PLD', 'GLW', 'SLV', 'GOLD', 'SCCO'
]

additional_tickers = [
    'ACN', 'ADSK', 'ALGN', 'AMZN', 'ANSS', 'AON', 'APD', 'BDX', 'BRK.B', 'BSX', 'BWA', 'CARR', 'CHTR', 'CME', 'COF', 'COP',
    'COST', 'CTAS', 'D', 'DD', 'DG', 'DOW', 'EMR', 'ETN', 'EXC', 'FANG', 'FDX', 'FIS', 'FITB', 'FMX', 'GD', 'GPN', 'HES', 'HLT',
    'HON', 'HSY', 'HUM', 'ICE', 'IDXX', 'ILMN'
]

all_tickers = list(set(Main_tickers + additional_tickers))
while len(all_tickers) < 150:
    all_tickers.append('EXTRA')
all_tickers = all_tickers[:150]
print(f"Total unique tickers: {len(all_tickers)}")
print("First 10 tickers:", all_tickers[:10])

Total unique tickers: 150
First 10 tickers: ['TXN', 'ECL', 'CAT', 'BDX', 'ORCL', 'UNH', 'NEM', 'GLW', 'SNPS', 'JPM']


In [27]:
def fetch_yf_data(ticker, start, end):
    try:
        df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=True)
        if df.empty:
            print(f"No data found for {ticker} in {start} to {end}")
            return None
        df = df.reset_index()
        df['Ticker'] = ticker
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        # Check if DataFrame has any valid data (at least 1 row)
        if len(df) > 0:
            return df[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']]
        else:
            print(f"No valid data rows for {ticker}")
            return None
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        return None

# Fetch training data (2021-2023)

In [28]:
train_data = []
failed_tickers_train = []
for ticker in all_tickers:
    df = fetch_yf_data(ticker, train_start, train_end)
    if df is not None:
        train_data.append(df)
    else:
        failed_tickers_train.append(ticker)
df_train = pd.concat(train_data, ignore_index=True) if train_data else pd.DataFrame()
df_train.to_csv("stock_data_train.csv", index=False)
print("Training data saved to stock_data_train.csv")
print(f"Failed tickers (training): {len(failed_tickers_train)}/{len(all_tickers)}")
print(f"Successful tickers: {len(train_data)}")


1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


No data found for BRK.B in 2021-01-01 to 2023-12-31



1 Failed download:
['GPS']: YFTzMissingError('possibly delisted; no timezone found')


No data found for GPS in 2021-01-01 to 2023-12-31



1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')


No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
No data found for EXTRA in 2021-01-01 to 2023-12-31
Training data saved to stock_data_train.csv
Failed tickers (training): 9/150
Successful tickers: 141


# Fetch testing data (2024)

In [29]:
test_data = []
failed_tickers_test = []
for ticker in all_tickers:
    df = fetch_yf_data(ticker, test_start, test_end)
    if df is not None:
        test_data.append(df)
    else:
        failed_tickers_test.append(ticker)
df_test = pd.concat(test_data, ignore_index=True) if test_data else pd.DataFrame()
df_test.to_csv("stock_data_test.csv", index=False)
print("Testing data saved to stock_data_test.csv")
print(f"Failed tickers (testing): {len(failed_tickers_test)}/{len(all_tickers)}")
print(f"Successful tickers: {len(test_data)}")


1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


No data found for BRK.B in 2024-01-01 to 2024-12-31



1 Failed download:
['GPS']: YFTzMissingError('possibly delisted; no timezone found')


No data found for GPS in 2024-01-01 to 2024-12-31



1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')


No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
No data found for EXTRA in 2024-01-01 to 2024-12-31
Testing data saved to stock_data_test.csv
Failed tickers (testing): 9/150
Successful tickers: 141


# Fetch forecast data (2025 up to today)

In [30]:
forecast_data = []
failed_tickers_forecast = []
current_date = datetime.today().strftime('%Y-%m-%d')  # April 7, 2025
for ticker in all_tickers:
    df = fetch_yf_data(ticker, forecast_start, current_date)
    if df is not None:
        forecast_data.append(df)
    else:
        failed_tickers_forecast.append(ticker)
df_forecast = pd.concat(forecast_data, ignore_index=True) if forecast_data else pd.DataFrame()
df_forecast.to_csv("stock_data_forecast.csv", index=False)
print(f"Forecast data saved to stock_data_forecast.csv (up to {current_date})")
print(f"Failed tickers (forecast): {len(failed_tickers_forecast)}/{len(all_tickers)}")
print(f"Successful tickers: {len(forecast_data)}")


1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


No data found for BRK.B in 2025-01-01 to 2025-04-08



1 Failed download:
['GPS']: YFTzMissingError('possibly delisted; no timezone found')


No data found for GPS in 2025-01-01 to 2025-04-08



1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EXTRA']: YFTzMissingError('possibly delisted; no timezone found')


No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
No data found for EXTRA in 2025-01-01 to 2025-04-08
Forecast data saved to stock_data_forecast.csv (up to 2025-04-08)
Failed tickers (forecast): 9/150
Successful tickers: 141
